In [279]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [280]:
columns=['language','text']
lista=list()

In [281]:
pages = {'en':'http://en.wikipedia.org/wiki/Wikipedia',
        'fi':'http://fi.wikipedia.org/wiki/Wikipedia',         
        'pl1':'https://pl.wikipedia.org/wiki/Podsumowanie_startów_zespołu_March_w_Formule_1',
         'pl2':'https://pl.wikipedia.org/wiki/Upiór_w_operze',
        'sv':'http://sv.wikipedia.org/wiki/Wikipedia',
         'da1':'https://da.wikipedia.org/wiki/VHL_2019-20',
         'da':'https://da.wikipedia.org/wiki/Kimbrere',
        'de':'http://de.wikipedia.org/wiki/Wikipedia',
        'nl':'http://nl.wikipedia.org/wiki/Wikipedia',
         'fi1':'https://fi.wikipedia.org/wiki/Pirjo_Honkasalo',
        'no':'http://no.wikipedia.org/wiki/Wikipedia', 
         'nl3':'https://nl.wikipedia.org/wiki/Berlin_Hauptbahnhof',
        'da2':'http://da.wikipedia.org/wiki/Wikipedia',
         'nl2':'https://nl.wikipedia.org/wiki/Spoorlijn_Amsterdam_-_Haarlem',
        'pl':'http://pl.wikipedia.org/wiki/Wikipedia',
        'da3':'https://da.wikipedia.org/wiki/Beryllium',         
         'fi2':'https://fi.wikipedia.org/wiki/Helsinki',
         'nl1':'https://nl.wikipedia.org/wiki/Geschiedenis_van_de_spoorwegen_in_Nederland',
         'no1':'https://no.wikipedia.org/wiki/Berlin',
         'sv1':'https://sv.wikipedia.org/wiki/Berlin',
         'pl3':'https://pl.wikipedia.org/wiki/Berlin'
        }

In [282]:
for element in pages:
    lang = element
    response = requests.get(pages[element])
    print(response.status_code)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content,'lxml')
        paragrs = soup.find_all('p')
        
        if paragrs:
            for pr in paragrs:
                tmptext = pr.text
                if len(tmptext) > 100:
                    
                    l=[lang,tmptext]
                    lista.append(l)
        

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [283]:
df = pd.DataFrame(lista,columns=columns)

In [284]:
def wytnij(text):
    return text[:2]
df['language'] = df['language'].apply(wytnij)

In [285]:
df.groupby('language').count()

,text
language,
da,140
de,159
en,148
fi,162
nl,109
no,152
pl,107
sv,217


In [286]:
import nltk
from nltk import word_tokenize
from string import punctuation

def textTokenize(text):
    table = word_tokenize(text)
    
    tab = [w for w in table if not (w in set(punctuation))]
    tab = [w for w in tab if w.isalpha()]
    
    #table = text.split(' ')
    return tab


In [287]:
df['token_text'] = df['text'].apply(textTokenize)


In [288]:
def cleanText(tokens):
    
    tmp = ' '.join(tokens)
    tmp = tmp.lower()
    
    return tmp
    

In [289]:
df['clean_text'] = df['token_text'].apply(cleanText)

In [290]:
##model training

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [291]:
df = sklearn.utils.shuffle(df)

In [292]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['language'], test_size=0.5)

In [293]:
vectorizer = TfidfVectorizer(ngram_range=(1,3),analyzer='char',use_idf=False)

In [294]:
clf = Pipeline([
    ('vect',vectorizer),
    ('clf',Perceptron()),
])

In [295]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='char', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=False,
                                 vocabulary=None)),
                ('clf',
                 Perceptron(alpha=0.0001, class_weight=None,
                            early_stopping=False, eta0=1.0, fit_intercept=True,
      

In [296]:
prediction = clf.predict(X_test)

In [297]:
metrics.accuracy_score(y_test,prediction)

0.9581239530988275

In [298]:
print(metrics.classification_report(y_test,prediction))

              precision    recall  f1-score   support

          da       0.88      0.98      0.93        66
          de       0.89      1.00      0.94        74
          en       0.99      0.99      0.99        85
          fi       0.99      0.99      0.99        76
          nl       1.00      0.98      0.99        53
          no       0.98      0.77      0.86        83
          pl       1.00      0.98      0.99        55
          sv       0.96      0.99      0.98       105

    accuracy                           0.96       597
   macro avg       0.96      0.96      0.96       597
weighted avg       0.96      0.96      0.96       597



In [299]:
clf.fit(df['clean_text'], df['language'])

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='char', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=False,
                                 vocabulary=None)),
                ('clf',
                 Perceptron(alpha=0.0001, class_weight=None,
                            early_stopping=False, eta0=1.0, fit_intercept=True,
      

In [319]:
clf.predict(['could you please confirm if swedac is an active customer i was told by satu sairi it was terminated but we are still being charged by exela this may contain confidential privileged information if you are not the intended recipient or have received this in error please notify the sender immediately and destroy this any unauthorized copying disclosure or distribution of the material in this is strictly forbidden'])

array(['en'], dtype='<U2')

In [313]:
df.to_excel('t.xlsx')

In [314]:
import pickle

In [315]:
file = open('langClf.pickle','wb')

In [317]:
pickle.dump(clf,file)

In [318]:
file.close()

In [323]:
w = clf.predict(['Dear user, Error description: The invoice e-mail message you created and sent could not be completed.Please re-send the corrected email message with all invoice attachments; the previously sent email has not reached its receiver.We have registered this error notification. Please use the subject field of this email message unaltered when sending email related to this error message.'])

In [324]:
w[0]

'en'

In [1]:
clf.predict(['Dear user, Error'])


NameError: name 'clf' is not defined